In [ ]:
import pandas as pd
import funzioni as fx
import numpy as np
import yfinance as yf
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
from multiprocessing import Pool, cpu_count, Manager, Value, Queue
import tensorflow as tf
from datetime import datetime, timedelta, date
import json
import os
import ctypes
from functools import partial
from typing import Tuple
import pickle

class Trade:
    def __init__(self, nome_ticker, data_inizio, data_fine, importo_da_investire, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione) -> None:
        self.SIMBOLO = nome_ticker
        self.DATA_INIZIO = data_inizio
        self.DATA_FINE = data_fine
        self.BILANCIO_INIZIALE = importo_da_investire
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self.log = None
        self.model = None
        self.valutazione_modello = None
        self.posizioni = None
        self.bilancio = None
        self.esito = None
        
    def avvia_trading(self, verbose=0) -> None:
        self.log = "\033[42m" + self.SIMBOLO + "\033[0m" + "\n"
        ticker = yf.download(self.SIMBOLO, start=self.DATA_INIZIO, end=self.DATA_FINE, progress=False)
        ticker.index = ticker.index.date
        str = f"Calcolo indicatori ticker {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"
        ticker = fx.crea_indicatori(ticker)
        ticker = fx.imposta_target(ticker)
        ticker.dropna(axis=0, inplace=True)

        str = f"Definizione features e target {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        idx, X, Y, _ = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, bilanciamento=0)
        str = f"Previsione {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        f1_score = F1Score(num_classes=1, average='macro', threshold=self.PROBABILITA_PER_ACQUISTO)
        self.model = load_model("model.h5", custom_objects={'Addons>F1Score': f1_score})
        y_pred = self.model.predict(X)
        self.valutazione_modello = self.model.evaluate(X, Y)

        X = X[:, -1, :]

        df_X = ticker.loc[ticker.index.intersection(idx)]
        y_pred = y_pred.reshape(-1,)
        df_y_pred = pd.DataFrame(y_pred, columns=['prob_True'], index=idx)
        self.dati = pd.concat([df_X, df_y_pred], axis=1)

        self.bilancio = self.BILANCIO_INIZIALE
        pos_aperta = False
        tp = sl = n_azioni = prezzo_acquisto = prezzo_tot = 0
        giorni_posizione = 0
        self.posizioni = pd.DataFrame(columns=["Bilancio", "Direzione", "Prezzo_un", "n_azioni", "Prezzo_tot", "TP", "SL", "Open", "High", "Low", "Esito", "P_L"], index=self.dati.index)

        for idx, row in self.dati.iterrows():
            if self.bilancio <= 0:
                break
            if pos_aperta == False:
                if row["prob_True"] > self.PROBABILITA_PER_ACQUISTO and self.bilancio > 0:
                    #COMPRA
                    prezzo_acquisto = row["Open"]
                    n_azioni = self.bilancio // prezzo_acquisto
                    prezzo_tot = n_azioni * prezzo_acquisto  
                    self.bilancio -= prezzo_tot          
                    tp = prezzo_acquisto * (1 + self.TP)
                    sl = prezzo_acquisto * (1 - self.SL)
                    pos_aperta = True
                    giorni_posizione = 0
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "COMPRA", "Prezzo_un": prezzo_acquisto, "n_azioni": n_azioni, "Prezzo_tot": -prezzo_tot, "TP": tp, "SL": sl, "Open": row["Open"], "High": row["High"], "Low": row["Low"],}
            else:
                if giorni_posizione == self.GIORNI_POS:
                    prezzo_tot_vendita = row['Close'] * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    if prezzo_tot_vendita > prezzo_tot:
                        esito = 'VINCITA'
                    else:
                        esito = 'PERDITA'
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": row['Close'], "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": esito, "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                if row["High"] >= tp:
                    prezzo_tot_vendita = tp * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": tp, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "VINCITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                elif row["Low"] <= sl:
                    prezzo_tot_vendita = sl * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": sl, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "PERDITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                else:
                    giorni_posizione += 1

        if pos_aperta: 
            self.bilancio += prezzo_tot

        tot_vincite = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "Esito"].count()
        tot_perdite = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "Esito"].count()
        max_vincita_value = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "P_L"].max(skipna=True)
        max_perdita_value = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "P_L"].min(skipna=True)
        max_vincita = int(max_vincita_value if max_vincita_value == max_vincita_value else 0)  # max_vincita_value == max_vincita_value è un modo per controllare se non è NaN
        max_perdita = int(max_perdita_value if max_perdita_value == max_perdita_value else 0)
        tot = int(fx.pct_change(self.BILANCIO_INIZIALE, self.bilancio))
        self.esito = (
            f"\nBilancio {self.SIMBOLO} = {round(self.bilancio, 2)}" + "\n" +
            f'Vincite: {tot_vincite}, vincita max: {max_vincita} %' + "\n" + 
            f'Perdite: {tot_perdite}, perdita max: {max_perdita} %' + "\n" +
            f'Totale: {tot} %'
        )
        if verbose == 1: print(self.esito)
        self.log += self.esito + "\n"

def _scarica(nome_simbolo, scarica_prima, scarica_dopo, data_inizio, data_fine, append):
    try:
        if append == True:
            ticker = pd.read_hdf(f'tickers/{nome_simbolo}.h5', "ticker")
            ti_min = ticker.index.min()
            ti_max = ticker.index.max()
            if scarica_prima:
                inizio = data_inizio - pd.Timedelta(days=365)
                fine = ti_min - pd.Timedelta(days=1)
                df_inizio = fx.analizza_ticker(nome_simbolo, start=inizio, end=fine, progress=False, dropna_iniziali=True, dropna_finali=False)
                ticker = pd.concat([df_inizio, ticker], axis=0, ignore_index=False)
            if scarica_dopo:
                inizio = ti_max - pd.Timedelta(days=365) 
                fine = data_fine
                df_fine = fx.analizza_ticker(nome_simbolo, start=inizio, end=fine, progress=False, dropna_iniziali=True, dropna_finali=False)
                ticker = ticker[ticker.index < df_fine.index.min()]
                ticker = pd.concat([ticker, df_fine], axis=0, ignore_index=False)
        else:
            ticker = fx.analizza_ticker(nome_simbolo, start=data_inizio, end=data_fine, progress=False, dropna_iniziali=True, dropna_finali=False)
        return nome_simbolo, ticker, ""
    except Exception as e:
        return nome_simbolo, None, str(e)

def _callback_tickers(result, totale_processati, tot_tickers):
    nome_simbolo, ticker, error = result
    if error == "":
        ticker.to_hdf(f'tickers/{nome_simbolo}.h5', key='ticker', mode='w')
    else:
        print(f"Errore su funzione di callback per {nome_simbolo}: {error}")

    with totale_processati.get_lock(): 
        totale_processati.value += 1
    print(f"{totale_processati.value}/{tot_tickers}) Completato scarico dati ticker {nome_simbolo}")   

def _carica_screener(nome_simbolo, lista_scr, prob):
    try:
        df = pd.read_hdf(f'screeners/{nome_simbolo}.h5', 'screener')
        df.index.set_names(['Data'], inplace=True)
        df['Ticker'] = nome_simbolo
        df.set_index('Ticker', append=True, inplace=True)
        df = df[df['Previsione'] > prob]
        lista_scr.append(df)
        return nome_simbolo, ""
    except Exception as e:
        return nome_simbolo, str(e)

def _carica_screener_callback(result, totale_processati, tot_tickers):
    nome_simbolo, error = result
    if error != "":
        print(f"Errore su funzione di callback per {nome_simbolo}: {error}")

    with totale_processati.get_lock(): 
        totale_processati.value += 1
    print(f"{totale_processati.value}/{tot_tickers}) Caricato su screener ticker {nome_simbolo}")   

class Posizione:
    def __init__(self, simbolo, ticker, data, n_azioni, bilancio, max_giorni_apertura, stop_loss, take_profit) -> None:
        self.simbolo = simbolo
        self.ticker = ticker
        self.data = data
        self.prezzo_unitario = ticker['Open'].iloc[0]
        self.n_azioni = n_azioni
        self.stop_loss = self.prezzo_unitario * (1 - stop_loss)
        self.take_profit = self.prezzo_unitario * (1 + take_profit)
        self.prezzo_tot = self.n_azioni * self.prezzo_unitario
        self.bilancio = bilancio - self.prezzo_tot 
        self.giorni_apertura = 1
        self.max_giorni_apertura = max_giorni_apertura
        self._colonne = ['Simbolo', 'Data', 'Tipo', 'Prezzo_unitario', 'n_azioni', 'Prezzo_tot', 'SL', 'TP', 'Bilancio', 'Giorni_apertura', 'Esito', 'Vincita', 'Perc']
        
    def to_df(self) -> pd.DataFrame:
        return pd.DataFrame([
            [self.simbolo, self.data.normalize(), 'COMPRA', self.prezzo_unitario.round(2), int(self.n_azioni), self.prezzo_tot.round(2), self.stop_loss.round(2), self.take_profit.round(2), self.bilancio.round(2), int(self.giorni_apertura), '', 0, 0]
        ], columns=self._colonne)

    def chiudi(self, data, bilancio, forza_chiusura=False):
        if data in self.ticker.index:
            dati_attuali = self.ticker[self.ticker.index == data]
            self.giorni_apertura += 1
            prezzo_attuale = dati_attuali['Open'].iloc[0]
            if (self.giorni_apertura > self.max_giorni_apertura) or (prezzo_attuale < self.stop_loss) or (prezzo_attuale > self.take_profit) or (forza_chiusura):
                prezzo_tot = prezzo_attuale * self.n_azioni
                bilancio += prezzo_tot
                if prezzo_attuale > self.prezzo_unitario:
                    esito = 'VINCITA'
                else:
                    esito = 'PERDITA'
                esito_tot = prezzo_tot - self.prezzo_tot
                esito_pct = fx.pct_change(self.prezzo_unitario, prezzo_attuale)
                return pd.DataFrame([
                    [self.simbolo, data.normalize(), 'VENDI', prezzo_attuale.round(2), int(self.n_azioni), prezzo_tot.round(2), self.stop_loss.round(2), self.take_profit.round(2), bilancio.round(2), int(self.giorni_apertura), esito, esito_tot.round(2), int(esito_pct)]
                ], columns=self._colonne)
            else:
                return None

class Borsa:
    def __init__(self, n_simboli_contemporanei, bilancio_iniziale, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione, data_inizio=pd.Timestamp(year=2005, month=1, day=1), data_fine=pd.Timestamp.now().normalize()):
        self.inizializza_gpu()

        self.N_SIMBOLI = n_simboli_contemporanei

        self.DATA_INIZIO = pd.to_datetime(data_inizio)
        while self.DATA_INIZIO.dayofweek >= 5:
            self.DATA_INIZIO += pd.Timedelta(days=1)

        self.DATA_FINE = pd.to_datetime(data_fine)
        while self.DATA_FINE.dayofweek >= 5:
            self.DATA_FINE -= pd.Timedelta(days=1)

        self.BILANCIO_INIZIALE = bilancio_iniziale
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self._posizioni = []
        self._bilancio = None
        self._data_corrente = None
        self._bilancio_per_simbolo = self.BILANCIO_INIZIALE / self.N_SIMBOLI
        self.esito_trading = None
        self.f1_score = F1Score(num_classes=1, average='macro', threshold=self.PROBABILITA_PER_ACQUISTO)
        self.model = load_model("model.h5", custom_objects={'Addons>F1Score': self.f1_score})
        self.lista_tickers = pd.read_parquet("lista_ticker.parquet")
        self.tot_tickers = len(self.lista_tickers)
        self.screener = pd.DataFrame()

        try:
            if os.path.exists(f'tickers/_indice.json'):
                with open(f'tickers/_indice.json', 'r') as jsonfile:
                    indice = json.load(jsonfile)
                prima_data = pd.to_datetime(indice['prima_data'])
                ultima_data = pd.to_datetime(indice['ultima_data'])
                
                if (self.DATA_INIZIO < prima_data):
                    scarica_prima = True
                else:
                    scarica_prima = False

                if (self.DATA_FINE > ultima_data):
                    scarica_dopo = True
                else:
                    scarica_dopo = False

                if scarica_prima or scarica_dopo:
                    self.scarica_tickers(scarica_prima, scarica_dopo, self.DATA_INIZIO, self.DATA_FINE, append=True)
                    self.avvia_screener(append=True)    
                    self.carica_screener()
                    self.screener.to_pickle('screeners/_screener.pickle')
                else:
                    self.screener = pd.read_pickle('screeners/_screener.pickle')
            else:
                self.scarica_tickers(scarica_prima=True, scarica_dopo=True, data_inizio=self.DATA_INIZIO, data_fine=self.DATA_FINE, append=False)
                self.avvia_screener(append=False)    
        except Exception as e:
            print(str(e))
        

    def inizializza_gpu(self):
        # Ottiene la lista di tutte le GPU fisiche disponibili
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                # Imposta TensorFlow per utilizzare tutte le GPU visibili
                tf.config.experimental.set_visible_devices(gpus, 'GPU')
                for gpu in gpus:
                    # Imposta TensorFlow per allocare dinamicamente la memoria su ogni GPU disponibile
                    tf.config.experimental.set_memory_growth(gpu, True)
            except RuntimeError as e:
                # L'eccezione viene sollevata se si tenta di impostare la visibilità delle GPU o la crescita della memoria
                # dopo che la sessione di TensorFlow è stata inizializzata
                print(e)

    def carica_screener(self):
        manager = Manager()
        lista_scr = manager.list()
        totale_processati = Value('i', 0)
        with Pool(cpu_count()) as p:
            callback_with_args = partial(_carica_screener_callback, totale_processati=totale_processati, tot_tickers=self.tot_tickers)
            for i in range(0, self.tot_tickers):
                nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                param = (nome_simbolo, lista_scr, self.PROBABILITA_PER_ACQUISTO)
                p.apply_async(_carica_screener, args=param, callback=callback_with_args)
            p.close()
            p.join()  
        self.screener = pd.concat(lista_scr, axis=0, ignore_index=False)
        self.screener = self.screener.sort_values(by=['Data', 'Previsione'], ascending=[True, False])

    def scarica_tickers(self, scarica_prima=True, scarica_dopo=True, data_inizio=pd.Timestamp(year=2005, month=1, day=1), data_fine=pd.Timestamp.now().normalize(), append=False) -> None: 
        tot_tickers = len(self.lista_tickers)
        totale_processati = Value('i', 0)
        with Pool(cpu_count()) as p:
            callback_with_args = partial(_callback_tickers, totale_processati=totale_processati, tot_tickers=tot_tickers)
            for i in range(0, tot_tickers):
                nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                param = (nome_simbolo, scarica_prima, scarica_dopo, data_inizio, data_fine, append)
                p.apply_async(_scarica, args=param, callback=callback_with_args)
            p.close()
            p.join()     

        indice = {
            'prima_data': self.DATA_INIZIO.strftime('%Y-%m-%d'),
            'ultima_data': self.DATA_FINE.strftime('%Y-%m-%d')
        }
        with open(f'tickers/_indice.json', 'w') as jsonfile:
            json.dump(indice, jsonfile, indent=4)    
 
    def avvia_screener(self, append=False) -> None:
        tot_tickers = len(self.lista_tickers)

        for i in range(0, tot_tickers):
            try:
                nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                print("\033[42m" + f'{i+1}/{tot_tickers}) Caricamento ticker {nome_simbolo}' + "\033[0m")
                ticker = pd.read_hdf(f'tickers/{nome_simbolo}.h5', 'ticker')
                if append:
                    scr = pd.read_hdf(f'screeners/{nome_simbolo}.h5', 'screener')
                    inizio = scr.index.max() - pd.Timedelta(days=365)
                    ticker_analisi = ticker[ticker.index >= inizio]
                    if scr.index.max() == ticker.index.max():
                        scr = scr.drop(scr.index[-1])
                    idx, X, Y, _ = fx.to_XY(ticker_analisi, bilanciamento=0)
                    print(f'Aggiornamento previsione {nome_simbolo}')
                    pred = self.model.predict(X)
                    scr_temp = pd.DataFrame({'Previsione': pred.flatten().round(2), 'Reale': Y.flatten()}, index=idx)
                    scr_temp = scr_temp[scr_temp.index > scr.index.max()]
                    scr = pd.concat([scr, scr_temp], axis=0, ignore_index=False)
                    print(f"Aggiornamento file {nome_simbolo}.h5")
                    scr.to_hdf(f'screeners/{nome_simbolo}.h5', key='screener', mode='w')
                else:
                    idx, X, Y, _ = fx.to_XY(ticker, bilanciamento=0)
                    print(f'Previsione {nome_simbolo}')
                    pred = self.model.predict(X)
                    scr = pd.DataFrame({'Previsione': pred.flatten().round(2), 'Reale': Y.flatten()}, index=idx)
                    scr = scr[scr.index >= self.DATA_INIZIO]
                    print(f"Salvataggio file {nome_simbolo}.h5")
                    scr.to_hdf(f'screeners/{nome_simbolo}.h5', key='screener', mode='w')
            except Exception as e:
                print(str(e))

    def ultimo_screener(self) -> (pd.Timestamp, pd.DataFrame):
        # Ottieni l'ultimo valore dell'indice di livello 0
        ultimo_indice = self.screener.index.get_level_values(0)[-1]
        # Usa .loc per selezionare tutte le righe con quell'indice di livello 0
        return ultimo_indice.date(), self.screener.loc[ultimo_indice]

    def avvia_trading(self) -> None:
        self._data_corrente = self.DATA_INIZIO
        self._posizioni = []
        self._bilancio = self.BILANCIO_INIZIALE
        self.esito_trading = pd.DataFrame()
        self._bilancio_per_simbolo = self.BILANCIO_INIZIALE / self.N_SIMBOLI
        while self._data_corrente <= self.DATA_FINE:
            print("\r\033[31m" + f'Data: {self._data_corrente.date()}, Pos. aperte: {len(self._posizioni)}' + "\033[0m", end=' ')
            self._chiudi_posizioni()
            if self._data_corrente in self.screener.index.get_level_values(0):
                scr = self.screener.loc[(self._data_corrente, slice(None)), :]
                i = 0
                esito = True
                while (len(self._posizioni) < self.N_SIMBOLI) and (i < len(scr)) and esito == True:
                    simbolo = scr.index[i][1]
                    esito = self._apri_posizione(simbolo)
                    i += 1
            self._data_corrente += timedelta(days=1)
        self._chiudi_posizioni(forza_chiusura=True)

    def _chiudi_posizioni(self, forza_chiusura=False):
        posizioni_da_mantenere = []
        for pos in self._posizioni:
            df = pos.chiudi(self._data_corrente, self._bilancio, forza_chiusura=forza_chiusura)
            if df is not None or forza_chiusura:
                self.esito_trading = pd.concat([self.esito_trading, df], axis=0, ignore_index=True)
                df = df.iloc[0]
                self._bilancio = df['Bilancio']
                pos_da_aprire = (self.N_SIMBOLI - len(self._posizioni))
                if pos_da_aprire == 0:
                    self._bilancio_per_simbolo = 0.
                else:
                    self._bilancio_per_simbolo = self._bilancio / pos_da_aprire
                print(f"VENDI {df['Simbolo']} n.{df['n_azioni']} azioni a {df['Prezzo_unitario']} € = {df['Prezzo_tot']} €, Esito: {df['Esito']}, Perc: {df['Perc']}, Bilancio: {round(self._bilancio, 2)}, Bilancio per simbolo: {round(self._bilancio_per_simbolo, 2)}")
            else:
                posizioni_da_mantenere.append(pos)
        self._posizioni = posizioni_da_mantenere

    def _apri_posizione(self, simbolo):
        try:
            ticker = pd.read_hdf(f'tickers/{simbolo}.h5', 'ticker')
            ticker = ticker[ticker.index >= self._data_corrente]
            prezzo = ticker["Open"].iloc[0]
            n_azioni = self._bilancio_per_simbolo // prezzo
            if (prezzo < self._bilancio_per_simbolo):
                pos = Posizione(simbolo, ticker, self._data_corrente, n_azioni, self._bilancio, self.GIORNI_POS, self.SL, self.TP)
                self._posizioni.append(pos)
                self.esito_trading = pd.concat([self.esito_trading, pos.to_df()], axis=0, ignore_index=True)
                self._bilancio = pos.bilancio
                pos_da_aprire = (self.N_SIMBOLI - len(self._posizioni))
                if pos_da_aprire == 0:
                    self._bilancio_per_simbolo = 0.
                else:
                    self._bilancio_per_simbolo = self._bilancio / pos_da_aprire
                print(f'COMPRA {simbolo} n.{pos.n_azioni} azioni a {pos.prezzo_unitario} € = {pos.prezzo_tot} €,  Bilancio: {round(self._bilancio, 2)}, Bilancio per simbolo: {round(self._bilancio_per_simbolo, 2)}')
                return True
            else:
                return False
        except Exception as e:
            print(f'Errore in apertura posizione: {str(e)}')
            return False



In [ ]:

if __name__ == "__main__":
    TP = 1
    SL = 0.01
    PROBABILITA_PER_ACQUISTO = 0.5
    BILANCIO_INIZIALE = 1000
    GIORNI_MAX_POSIZIONE = 20
    N_SIMBOLI = 10

    borsa = Borsa(N_SIMBOLI, BILANCIO_INIZIALE, PROBABILITA_PER_ACQUISTO, SL, TP, GIORNI_MAX_POSIZIONE)


In [ ]:
borsa.avvia_trading()

In [ ]:
borsa.esito_trading.to_excel('esito.xlsx')